## Install All the Required Packages

In [1]:
# installing modules this way is not recommended because it installs it globally. Better to use requirements.txt
!pip install langchain
!pip install pinecone-client
!pip install pypdf

In [2]:
!pip install openai
!pip install tiktoken

## Import All the Required Libraries

In [43]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

## Load the PDF Files

In [4]:
!mkdir pdfs

A subdirectory or file pdfs already exists.


In [5]:
# Have to pip install gdown before this
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf
# It errors with 'wget' is not recognized as an internal or external command, operable program or batch file. It might work on linux/mac. If windows, see below.


'wget' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "c:\swe\code\Complete-Generative-AI-Course-on-YouTube\.venv\Scripts\gdown-script.py", line 33, in <module>
    sys.exit(load_entry_point('gdown==2.3.1', 'console_scripts', 'gdown')())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\swe\code\Complete-Generative-AI-Course-on-YouTube\.venv\Lib\site-packages\gdown\__init__.py", line 72, in main
    while (count < max_trial) and (os.stat(filename).st_size < 100000):
                                   ^^^^^^^^^^^^^^^^^
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'pdfs/yolov7paper.pdf'
'wget' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "c:\swe\code\Complete-Generative-AI-Course-on-YouTube\.venv\Scripts\gdown-script.py", line 33, in <module>
    sys.exit(load_

In Windows, in WSL
cd into /mnt/c/swe/code/Complete-Generative-AI-Course-on-YouTube
then

.venv/Scripts/gdown.exe 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf

But it errors with 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'pdfs/yolov7paper.pdf'

Maybe the pdfs are only available to the tutor, so just use your own pdf, like a resume or something, and put it in the pdfs folder.

## Extract the Text from the PDF's

In [6]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [7]:
data # list of "documents". each page in a pdf is a "document"

[Document(page_content='A n d r e w\nB u e n s a l i d a\nPortfolio:\nwww .anhonestobserver .com\n●\n661-964-9268\n●\nSan\nDiego,\nCA \nandrewcbuensalida@gmail.com\n●\nlinkedin.com/in/andrewcbuensalida\n●\ngithub.com/andrewcbuensalida\nSummary\nFormer\nDepartment\nof\nHomeland\nSecurity\nLead,\nCurrent\nHigh-producing,\nCertified\nFull-Stack\nDevOps,\nReact, \nNodeJs,\nA WS\nSolutions\nArchitect,\nand\nDatabase\nSoftware\nEngineer\nwith\nover\n100+\nproduction\ngrade\napps\nand\nover \n4\nyears\nof\nexperience\nT echnical\nSkills\nAmazon\nW eb\nServices,\nAndroid,\nApache\nAirflow ,\nA ws\nAmplify ,\nA ws\nApi\nGateway ,\nA ws\nAurora,\nA ws\nCertificate\nManager , \nA ws\nCloudfront,\nA ws\nCode\nPipeline,\nA ws\nDynamodb,\nA ws\nEc2,\nA ws\nEcr ,\nA ws\nEcs,\nA ws\nEks,\nA ws\nElastic\nBeanstalk,\nA ws \nLambda,\nA ws\nLoad-balancer ,\nA ws\nRoute\n53,\nA ws\nS3,\nA ws\nSam,\nA ws\nSns,\nA ws\nVpc,\nAxios,\nBeautiful\nSoup,\nBootstrap, \nChakra-ui,\nCi/cd,\nCryptojs,\nD3,\nDbeaver ,\

In [72]:
len(data)

11

## Split the Extracted Data into Text Chunks

In [81]:
text_splitter = RecursiveCharacterTextSplitter(
    # chunk_size=500,
    # chunk_overlap=20,
    separators=[
        "c, CE-IVD (Europe"
    ],  # this doesn't seem to work the way that I expected. According to https://dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846, It takes in the large text then tries to split it by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. If it is still larger than our specified chunk size it moves to the next character in the set until we get a split that is less than our specified chunk size. It may not be splitting the text at "c, CE-IVD (Europe" because the chunk could fit within the default chunk_size.
    keep_separator=True,
)  # this needs to split the data by pdf, not chunk_size

In [82]:
text_chunks = text_splitter.split_documents(data)
text_chunks

[Document(page_content='A n d r e w\nB u e n s a l i d a\nPortfolio:\nwww .anhonestobserver .com\n●\n661-964-9268\n●\nSan\nDiego,\nCA \nandrewcbuensalida@gmail.com\n●\nlinkedin.com/in/andrewcbuensalida\n●\ngithub.com/andrewcbuensalida\nSummary\nFormer\nDepartment\nof\nHomeland\nSecurity\nLead,\nCurrent\nHigh-producing,\nCertified\nFull-Stack\nDevOps,\nReact, \nNodeJs,\nA WS\nSolutions\nArchitect,\nand\nDatabase\nSoftware\nEngineer\nwith\nover\n100+\nproduction\ngrade\napps\nand\nover \n4\nyears\nof\nexperience\nT echnical\nSkills\nAmazon\nW eb\nServices,\nAndroid,\nApache\nAirflow ,\nA ws\nAmplify ,\nA ws\nApi\nGateway ,\nA ws\nAurora,\nA ws\nCertificate\nManager , \nA ws\nCloudfront,\nA ws\nCode\nPipeline,\nA ws\nDynamodb,\nA ws\nEc2,\nA ws\nEcr ,\nA ws\nEcs,\nA ws\nEks,\nA ws\nElastic\nBeanstalk,\nA ws \nLambda,\nA ws\nLoad-balancer ,\nA ws\nRoute\n53,\nA ws\nS3,\nA ws\nSam,\nA ws\nSns,\nA ws\nVpc,\nAxios,\nBeautiful\nSoup,\nBootstrap, \nChakra-ui,\nCi/cd,\nCryptojs,\nD3,\nDbeaver ,\

In [78]:
len(text_chunks)

11

In [12]:
text_chunks[1]

Document(page_content='Android,\nApache\nAirflow ,\nA ws\nAmplify ,\nA ws\nApi\nGateway ,\nA ws\nAurora,\nA ws\nCertificate\nManager , \nA ws\nCloudfront,\nA ws\nCode\nPipeline,\nA ws\nDynamodb,\nA ws\nEc2,\nA ws\nEcr ,\nA ws\nEcs,\nA ws\nEks,\nA ws\nElastic\nBeanstalk,\nA ws \nLambda,\nA ws\nLoad-balancer ,\nA ws\nRoute\n53,\nA ws\nS3,\nA ws\nSam,\nA ws\nSns,\nA ws\nVpc,\nAxios,\nBeautiful\nSoup,\nBootstrap, \nChakra-ui,\nCi/cd,\nCryptojs,\nD3,\nDbeaver ,\nDjango,\nDocker ,\nEjs,\nElectronjs,\nEnzyme,\nExpress,\nFirebase,\nFlask,\nFormik,\nGit, \nGithub\nActions,\nGoogle\nAds,', metadata={'source': 'pdfs\\Andrew_Buensalida_Resume.pdf', 'page': 0})

In [13]:
text_chunks[2]

Document(page_content='Google\nAds,\nGoogle\nAssistant,\nGoogle\nCloud\nPlatform,\nGraphql,\nHandlebars,\nHeroku,\nHostgator ,\nHtml/css, \nJava,\nJavascript,\nJenkins,\nJest,\nJquery ,\nJwt,\nKubernetes,\nLinux,\nLucid\nChart,\nLucidchart,\nMaterial\nUi,\nMaven,\nMinio, \nMongodb,\nMongoose,\nMs\nSql\nServer ,\nMvc,\nMysql,\nNeo4j,\nNestJs,\nNginx,\nNgrok,\nNodejs,\nNodemailer ,\nOracle\nDb,\nPdf-lib, \nPhp,\nPostgres,\nPuppeteer ,\nPython,\nRabbitmq,\nReact\nFinancial\nCharts,\nReact\nHook\nForm,\nReact\nNative,\nReact\nNavigation, \nReact\nRouter ,\nReact\nT able,\nReact', metadata={'source': 'pdfs\\Andrew_Buensalida_Resume.pdf', 'page': 0})

In [14]:
text_chunks[3]

Document(page_content='React\nT able,\nReact\nT esting\nLibrary ,\nReact,\nRedis,\nRedux,\nResponsive,\nScss,\nSelenium,\nSequelize,\nServerless \nFramework,\nSinon,\nSocket.io,\nSpring\nBoot,\nSqlite,\nSsl,\nStripe,\nSwagger ,\nT ailwind,\nT wilio,\nT ypeorm,\nT ypescript,\nUser \nAuthentication,\nV isio,\nV ue,\nW eb\nScraping,\nW ebsockets,\nW ordpress,\nY up,\nand\nmore…\nProfessional\nExperience\nPersonal\nProjects\n–\nSoftware\nEngineer .\nSan\nDiego,\nCA\n5/23-Present \n●\nT ook\ninitiative\nin\nbuilding\nand\ndeploying\nmultiple\napps\nwith\na\nvariety\nof\ntech', metadata={'source': 'pdfs\\Andrew_Buensalida_Resume.pdf', 'page': 0})

## Download the Embeddings

In [15]:
import os

# this is not needed if you have the API key in a .env file
# os.environ["OPENAI_API_KEY"] = "your openai api key here"

In [16]:
embeddings = OpenAIEmbeddings()

In [17]:
result = embeddings.embed_query("How are you!")
result

[-0.0048034521751105785,
 -0.009349464438855648,
 -0.0030390468891710043,
 -0.026070501655340195,
 -0.016074297949671745,
 0.012087118811905384,
 0.0011380728101357818,
 -0.013499898836016655,
 -0.019389621913433075,
 -0.0034032303374260664,
 0.028330950066447258,
 0.0067499494180083275,
 -0.013939430005848408,
 -0.006392044946551323,
 0.0075850593857467175,
 -0.022780293598771095,
 0.027502119541168213,
 -0.002351494040340185,
 0.01873660460114479,
 -0.02715049497783184,
 0.002064228756353259,
 0.02185099944472313,
 0.008646214380860329,
 -0.007001110352575779,
 -0.0001958270149771124,
 0.0002062266576103866,
 0.011189218610525131,
 -0.005434493999928236,
 0.022428669035434723,
 -0.027753280475735664,
 0.011660145595669746,
 -0.0006844134768471122,
 0.0008178427233360708,
 0.00273451441898942,
 0.010994568467140198,
 -0.02306913025677204,
 0.0007103144307620823,
 0.007760872133076191,
 0.032751381397247314,
 -0.0006749949534423649,
 0.025605853646993637,
 -0.014240823686122894,
 0.002

In [18]:
len(result)

1536

## Initializing the Pinecone

In [22]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')


In [30]:
# from pinecone import Pinecone # This is not needed since we are using the pinecone from langchain
# pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mytestindex" # put in the name of your pinecone index here. When creating the index in pinecone.io, the Dimensions have to be the same as the result length. 


## Create Embeddings for each of the Text Chunk

In [29]:
# embeddings is from openai. This uploads the embeddings to your index in pinecone.io. I guess Pinecone automatically uses the API key from .env file.
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)
docsearch

## If you already have an index, you can load it like this

In [32]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch

## Similarity Search

In [41]:
query = "Who is the best software engineer out of the candidates?"

In [38]:
docs = docsearch.similarity_search(query, k=3)
docs # this is a list of chunks

page_content='and India.\n- Used Java, Spring, Hibernate, RPC, SOA, JSP, MVC\nDeveloper Portal - Maintained the application ingestion workflow at\ndeveloper.amazon.com/appsandservices. Team size of 20\nincluding 2 QA engineers. Worked with remote teams in Seattle and India.\n- Used Java, Spring, Hibernate, RPC, SOA, JSP, MVC\nL-3 Communications\nSoftware Engineer Intern\nJune 2011\xa0-\xa0September 2012\xa0 (1 year 4 months)\nSan Diego, CA\nDesigned and devloped frontend applications for interacting with Matlab/'
page_content='Node,\nDynamoDB,\nSNS,\nOracle\nPL/SQL, \nS3,\nSASS,\nBootstrap,\nRedux,\nServerless\nFramework,\nPDF-lib,\nNode\nmailer ,\nJest,\nSinon,\nJenkins/Cloudbees, \nGithub,\nJira,\nLucid\nchart,\nV isio,\nElectronJs,\nand\nmore. \n●\nT aught\njunior\ndevelopers\nbest\npractices\nsuch\nas\nDR Y ,\nnaming\nconventions,\nDevOps\nbranching\nstrategies, \ndebugging,\ndocumentation,\nsystem\ndesign\ndiagrams\netc\nto\nachieve\nthe\nhighest\ncoding\nstandard'
page_content='•

## Creating a LLM Model Wrapper

In [44]:
llm = OpenAI()

In [45]:
# create the chain to answer questions. llm is openai. chain_type "stuff" means to fill, meaning it takes a list of documents and inserts them into a prompt to pass to the llm. This chain is well-suited for applications where documents are small and only a few are passed in for most calls. retriever is the pinecone vectordb.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

## Q/A

In [50]:
query = "Which candidates have experience in python"

In [52]:
qa.invoke(query)

{'query': 'Which candidates have experience in python',
 'result': '\n\nIt is not possible to determine which candidates have experience in Python based on the given context. Some potential candidates who may have experience in Python include the software engineer intern at L-3 Communications and the developer at Google. However, without further information it is not possible to accurately determine their experience in Python.'}

In [55]:
query = "What is Andrew Buensalida's experience"

In [56]:
qa.invoke(query) # hallucinates. It says Andrew Buensalida was a Director or Business Development at Libyl Inc. and a Senior Software Engineer at Agena Bioscience. Sarah Uran is the Director of Business Development at Libyl Inc. Matthew Borger is the Senior Software Engineer at Agena Bioscience. Need to some how seperate the chunks based on pdf, not word count.

{'query': "What is Andrew Buensalida's experience",
 'result': " Andrew Buensalida's experience includes working as a former Department of Homeland Security Lead, current High-producing, Certified Full-Stack DevOps, React, NodeJs, AWS Solutions Architect, and Database Software Engineer with over 100+ production grade apps and over 4 years of experience. He also has experience as a Director of Business Development at Libyl Inc. and as a Senior Software Engineer at Agena Bioscience."}

In [83]:
import sys

In [40]:
# This is just to make it a cli app.
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what is yolo v7
Answer:  YOLOv7 is a real-time object detector which surpasses all known object detectors in both speed and accuracy. It has the highest accuracy of 56.8% AP among all known detectors and can run from 5 FPS to 160 FPS.
Input Prompt: tell me about Rechel Green
Answer:  Rachel Green is a PhD in English from the University of Illinois at Urbana-Champaign. Her dissertation title was “Down on the Farm: World War One and the Emergence of Literary Modernism in the American South.” She also has a MA in English and was awarded a Summer Research Grant, a Graduate College Conference Travel Grant, Most Outstanding Butler Woman, and an Academic Scholarship. She has published extensively and has given multiple conference presentations.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
